# Baseline models: seasonal Naive, weighted Naive

In [18]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from math import sqrt
import itertools
from itertools import product

import warnings
warnings.filterwarnings("ignore")

## Load data

The dataset includes Nordecon AS consolidated quarterly sales data for 18 years from Q1 2005 to Q4 2023.

In [19]:
# Load the CSV file
df = pd.read_csv('NCN_sales_Q.csv', parse_dates=True)

# Convert the 'quarter' column to datetime format
df['quarter'] = pd.to_datetime(df['quarter'])

# Set the 'quarter' column as the index
df.set_index('quarter', inplace=True)

# Resample the data to represent the end of each quarter
df_resampled = df.resample('Q').sum()

# Optionally, you can change the index format to YYYY-QQ
df_resampled.index = df_resampled.index.to_period('Q')

# Display the resulting Series
series = df_resampled.squeeze("columns")
series


quarter
2005Q1    18702
2005Q2    22122
2005Q3    34660
2005Q4    24577
2006Q1    20840
          ...  
2022Q4    79376
2023Q1    46742
2023Q2    76021
2023Q3    75548
2023Q4    73766
Freq: Q-DEC, Name: NCN_sales_est, Length: 76, dtype: int64

## Separate dataset for training and validation

In [20]:
# Number of quarters to keep for validation
validation_quarters = 4

splits_val = {'train': [], 'val': []}

for i in range(len(series)-19, len(series) - validation_quarters, 1):
    train_data = series.iloc[:i + validation_quarters]
    valid_data = series.iloc[i + validation_quarters:i + validation_quarters + validation_quarters]
    print(f'TRAIN: {train_data.index}, VALID: {valid_data.index}')
    splits_val['train'].append(train_data)
    splits_val['val'].append(valid_data)

    if i + 11 >= len(series) - 1:
        break

TRAIN: PeriodIndex(['2005Q1', '2005Q2', '2005Q3', '2005Q4', '2006Q1', '2006Q2',
             '2006Q3', '2006Q4', '2007Q1', '2007Q2', '2007Q3', '2007Q4',
             '2008Q1', '2008Q2', '2008Q3', '2008Q4', '2009Q1', '2009Q2',
             '2009Q3', '2009Q4', '2010Q1', '2010Q2', '2010Q3', '2010Q4',
             '2011Q1', '2011Q2', '2011Q3', '2011Q4', '2012Q1', '2012Q2',
             '2012Q3', '2012Q4', '2013Q1', '2013Q2', '2013Q3', '2013Q4',
             '2014Q1', '2014Q2', '2014Q3', '2014Q4', '2015Q1', '2015Q2',
             '2015Q3', '2015Q4', '2016Q1', '2016Q2', '2016Q3', '2016Q4',
             '2017Q1', '2017Q2', '2017Q3', '2017Q4', '2018Q1', '2018Q2',
             '2018Q3', '2018Q4', '2019Q1', '2019Q2', '2019Q3', '2019Q4',
             '2020Q1'],
            dtype='period[Q-DEC]', name='quarter'), VALID: PeriodIndex(['2020Q2', '2020Q3', '2020Q4', '2021Q1'], dtype='period[Q-DEC]', name='quarter')
TRAIN: PeriodIndex(['2005Q1', '2005Q2', '2005Q3', '2005Q4', '2006Q1', '2006Q2',
       

## Separate test set

In [21]:
# Number of quarters to keep for validation
validation_quarters = 4

splits_test = {'train': [], 'test': []}

for i in range(len(series)-15, len(series) - validation_quarters, 1):
    train_data = series.iloc[:i + validation_quarters]
    valid_data = series.iloc[i + validation_quarters:i + validation_quarters + validation_quarters]
    print(f'TRAIN: {train_data.index}, TEST: {valid_data.index}')
    splits_test['train'].append(train_data)
    splits_test['test'].append(valid_data)

    if i + 7 >= len(series) - 1:
        break

TRAIN: PeriodIndex(['2005Q1', '2005Q2', '2005Q3', '2005Q4', '2006Q1', '2006Q2',
             '2006Q3', '2006Q4', '2007Q1', '2007Q2', '2007Q3', '2007Q4',
             '2008Q1', '2008Q2', '2008Q3', '2008Q4', '2009Q1', '2009Q2',
             '2009Q3', '2009Q4', '2010Q1', '2010Q2', '2010Q3', '2010Q4',
             '2011Q1', '2011Q2', '2011Q3', '2011Q4', '2012Q1', '2012Q2',
             '2012Q3', '2012Q4', '2013Q1', '2013Q2', '2013Q3', '2013Q4',
             '2014Q1', '2014Q2', '2014Q3', '2014Q4', '2015Q1', '2015Q2',
             '2015Q3', '2015Q4', '2016Q1', '2016Q2', '2016Q3', '2016Q4',
             '2017Q1', '2017Q2', '2017Q3', '2017Q4', '2018Q1', '2018Q2',
             '2018Q3', '2018Q4', '2019Q1', '2019Q2', '2019Q3', '2019Q4',
             '2020Q1', '2020Q2', '2020Q3', '2020Q4', '2021Q1'],
            dtype='period[Q-DEC]', name='quarter'), TEST: PeriodIndex(['2021Q2', '2021Q3', '2021Q4', '2022Q1'], dtype='period[Q-DEC]', name='quarter')
TRAIN: PeriodIndex(['2005Q1', '2005Q2', '2005Q3'

In [22]:
# First validation split
splits_val['val'][0]

quarter
2020Q2    73062
2020Q3    70453
2020Q4    50625
2021Q1    46978
Freq: Q-DEC, Name: NCN_sales_est, dtype: int64

In [23]:
# First test split
splits_test['test'][0]

quarter
2021Q2    65862
2021Q3    84815
2021Q4    74396
2022Q1    64830
Freq: Q-DEC, Name: NCN_sales_est, dtype: int64

## sNaive (same last quarter)

### Validation

In [24]:
valid_MAPE = []
valid_R2 = []
valid_RMSE = []
snaive_cv_preds = []
actuals = []

f = 1
# Loop for naive forecasting and calculating RMSE and MAPE
for train_data, valid_data in zip(splits_val['train'], splits_val['val']):
  print('Fold: ', f)
  # walk-forward validation
  history = [x for x in train_data]
  predictions = list()
  for i in range(len(valid_data)):
 	  # predict
 	  yhat = history[-4]
 	  predictions.append(yhat)
 	  # observation
 	  obs = valid_data[i]
 	  history.append(obs)
 	  print('>Predicted=%.3f, Expected=%3.f' % (yhat, obs))
    # report performance
  rmse_snaive = sqrt(mean_squared_error(valid_data, predictions))
  mape_snaive = mean_absolute_percentage_error(valid_data, predictions)
  r2_snaive = r2_score(valid_data, predictions)
  valid_MAPE.append(mape_snaive)
  valid_R2.append(r2_snaive)
  valid_RMSE.append(rmse_snaive)
  snaive_cv_preds.append(predictions)
  actuals.append(valid_data.values)

  f += 1
  print('MAPE: %.3f' % mape_snaive)
  print('RMSE: %.3f' % rmse_snaive)
  print('R2: %.3f' % r2_snaive)
  print('')
RMSE_mean_snaive_valid = round(np.mean(valid_RMSE), 4)
MAPE_mean_snaive_valid = round(np.mean(valid_MAPE), 4)
MAPE_std_snaive_valid = round(np.std(valid_MAPE), 4)
R2_mean_snaive_valid = round(np.mean(valid_R2), 4)
print(f'Mean validation MAPE is {MAPE_mean_snaive_valid}.')
print(f'Validation MAPE std is {MAPE_std_snaive_valid}.')
print(f'Mean validation RMSE is {RMSE_mean_snaive_valid}.')
print(f'Mean validation R2 is {R2_mean_snaive_valid}.')

Fold:  1
>Predicted=61601.000, Expected=73062
>Predicted=66551.000, Expected=70453
>Predicted=53727.000, Expected=50625
>Predicted=47534.000, Expected=46978
MAPE: 0.071
RMSE: 6255.231
R2: 0.709

Fold:  2
>Predicted=66551.000, Expected=70453
>Predicted=53727.000, Expected=50625
>Predicted=47534.000, Expected=46978
>Predicted=73062.000, Expected=65862
MAPE: 0.059
RMSE: 4387.401
R2: 0.804

Fold:  3
>Predicted=53727.000, Expected=50625
>Predicted=47534.000, Expected=46978
>Predicted=73062.000, Expected=65862
>Predicted=70453.000, Expected=84815
MAPE: 0.088
RMSE: 8185.942
R2: 0.699

Fold:  4
>Predicted=47534.000, Expected=46978
>Predicted=73062.000, Expected=65862
>Predicted=70453.000, Expected=84815
>Predicted=50625.000, Expected=74396
MAPE: 0.153
RMSE: 14348.141
R2: -0.069

Fold:  5
>Predicted=73062.000, Expected=65862
>Predicted=70453.000, Expected=84815
>Predicted=50625.000, Expected=74396
>Predicted=46978.000, Expected=64830
MAPE: 0.218
RMSE: 16895.720
R2: -3.423

Fold:  6
>Predicted=7

### Test set

In [25]:
test_MAPE = []
test_R2 = []
test_RMSE = []
snaive_test_preds = []
actuals = []

f = 1
# Loop for naive forecasting and calculating RMSE and MAPE
for train_data, test_data in zip(splits_test['train'], splits_test['test']):
  print('Fold: ', f)
  # walk-forward validation
  history = [x for x in train_data]
  predictions = list()
  for i in range(len(test_data)):
 	  # predict
 	  yhat = history[-4]
 	  predictions.append(yhat)
 	  # observation
 	  obs = test_data[i]
 	  history.append(obs)
 	  print('>Predicted=%.3f, Expected=%3.f' % (yhat, obs))
    # report performance
  rmse_snaive = sqrt(mean_squared_error(test_data, predictions))
  mape_snaive = mean_absolute_percentage_error(test_data, predictions)
  r2_snaive = r2_score(test_data, predictions)
  test_MAPE.append(mape_snaive)
  test_R2.append(r2_snaive)
  test_RMSE.append(rmse_snaive)
  snaive_test_preds.append(predictions)
  actuals.append(test_data.values)

  f += 1
  print('MAPE: %.3f' % mape_snaive)
  print('RMSE: %.3f' % rmse_snaive)
  print('R2: %.3f' % r2_snaive)
  print('')
RMSE_mean_snaive_test = round(np.mean(test_RMSE), 4)
MAPE_mean_snaive_test = round(np.mean(test_MAPE), 4)
MAPE_std_snaive_test = round(np.std(test_MAPE), 4)
R2_mean_snaive_test = round(np.mean(test_R2), 4)
print(f'Mean test MAPE is {MAPE_mean_snaive_test}.')
print(f'Test MAPE std is {MAPE_std_snaive_test}.')
print(f'Mean test RMSE is {RMSE_mean_snaive_test}.')
print(f'Mean test R2 is {R2_mean_snaive_test}.')

Fold:  1
>Predicted=73062.000, Expected=65862
>Predicted=70453.000, Expected=84815
>Predicted=50625.000, Expected=74396
>Predicted=46978.000, Expected=64830
MAPE: 0.218
RMSE: 16895.720
R2: -3.423

Fold:  2
>Predicted=70453.000, Expected=84815
>Predicted=50625.000, Expected=74396
>Predicted=46978.000, Expected=64830
>Predicted=65862.000, Expected=79817
MAPE: 0.235
RMSE: 17921.798
R2: -4.850

Fold:  3
>Predicted=50625.000, Expected=74396
>Predicted=46978.000, Expected=64830
>Predicted=65862.000, Expected=79817
>Predicted=84815.000, Expected=85235
MAPE: 0.194
RMSE: 16421.577
R2: -3.748

Fold:  4
>Predicted=46978.000, Expected=64830
>Predicted=65862.000, Expected=79817
>Predicted=84815.000, Expected=85235
>Predicted=74396.000, Expected=79376
MAPE: 0.129
RMSE: 11601.861
R2: -1.350

Fold:  5
>Predicted=65862.000, Expected=79817
>Predicted=84815.000, Expected=85235
>Predicted=74396.000, Expected=79376
>Predicted=64830.000, Expected=46742
MAPE: 0.157
RMSE: 11692.889
R2: 0.409

Fold:  6
>Predic

In [26]:
# Calculate absolute percentage errors
absolute_percentage_errors = []

for preds, actual in zip(snaive_test_preds, actuals):
    abs_percent_errors = [abs((pred - true) / true) for pred, true in zip(preds, actual)]
    absolute_percentage_errors.append(abs_percent_errors)

In [27]:
mean_values = [np.mean(member) for member in zip(*absolute_percentage_errors)]
mean_values

[0.1878773278197889,
 0.18045408841835758,
 0.1753153211306582,
 0.14488174996642855]

### Results

In [28]:
data = {
    'sNaive valid': [MAPE_mean_snaive_valid, MAPE_std_snaive_valid, RMSE_mean_snaive_valid,  R2_mean_snaive_valid, '***'],
    'sNaive test': [MAPE_mean_snaive_test, MAPE_std_snaive_test, RMSE_mean_snaive_test,  R2_mean_snaive_test, mean_values]}

df_results = pd.DataFrame(data, index=['Mean MAPE', 'MAPE std', 'Mean RMSE', 'Mean R2', 'FH MAPEs'])
df_results

,sNaive valid,sNaive test
Mean MAPE,0.1434,0.1721
MAPE std,0.0633,0.0372
Mean RMSE,12002.2089,13204.6307
Mean R2,-1.4037,-1.4762
FH MAPEs,***,"[0.1878773278197889, 0.18045408841835758, 0.17..."


## Weighted Naive (same quarters last three years)

### Validation

In [29]:
best_mape_overall = float('inf')
best_std_overall = None
best_weights_overall = None
best_rmse_overall = None
best_r2_overall = None
best_combined_forecasts = None

# Loop over all possible weight combinations
for weights in product(range(1, 11), repeat=3):  # Assuming 3 forecasts in forecast_lists
    weights = np.array(weights) / sum(weights)  # Normalize weights to sum to 1.0
    mean_mape_across_folds = 0

    f = 1
    mape_fold_values = []  # To store MAPE values for each fold
    combined_forecasts = []  # To store combined forecasts for each fold

    # Loop for naive forecasting and calculating RMSE and MAPE for each fold
    for train_data, valid_data in zip(splits_val['train'], splits_val['val']):
        print('Fold: ', f)
        history = [x for x in train_data]
        pred_1 = []
        pred_2 = []
        pred_3 = []
        actual = []  # Initialize the actual list here

        for i in range(len(valid_data)):
            obs = valid_data[i]
            actual.append(obs)  # Append actual values here

        for i in range(-4, 0, 1):
            p_1 = history[i]
            p_2 = history[i - 4]
            p_3 = history[i - 8]
            pred_1.append(p_1)
            pred_2.append(p_2)
            pred_3.append(p_3)

        forecast_lists = np.vstack((pred_1, pred_2, pred_3))

        combined_forecast = np.dot(weights, forecast_lists)
        combined_forecasts.append(combined_forecast)

        rmse_wnaive = sqrt(mean_squared_error(actual, combined_forecast))
        mape_wnaive = mean_absolute_percentage_error(actual, combined_forecast)
        r2_wnaive = r2_score(actual, combined_forecast)

        print("MAPE:", mape_wnaive)
        print("RMSE:", rmse_wnaive)
        print("R2:", r2_wnaive)

        # Update lowest MAPE for the current fold
        mape_fold_values.append(mape_wnaive)

        f += 1

    # Calculate mean MAPE across all folds for the current set of weights
    mean_mape_across_folds = np.mean(mape_fold_values)
    mean_std_across_folds = np.std(mape_fold_values)

    # Update overall best values if the current mean MAPE is better
    if mean_mape_across_folds < best_mape_overall:
        best_mape_overall = mean_mape_across_folds
        best_std_overall = mean_std_across_folds
        best_rmse_overall = rmse_wnaive
        best_r2_overall = r2_wnaive
        best_weights_overall = weights
        best_combined_forecasts = np.array(combined_forecasts)  # Convert to numpy array

    print("Mean MAPE across folds for current weights:", mean_mape_across_folds)
    print("Current Weights:", weights)
    print('')

# Print overall best values
print("Best Mean MAPE across all folds:", best_mape_overall)
print("Corresponding MAPE std across all folds:", best_std_overall)
print("Corresponding Mean RMSE across all folds:", best_rmse_overall)
print("Corresponding Mean R2 across all folds:", best_r2_overall)
print("Best Weights across all folds:", best_weights_overall)

Streaming output truncated to the last 5000 lines.
RMSE: 9972.792774639303
R2: 0.5532403839269034
Fold:  4
MAPE: 0.16815130724458166
RMSE: 14995.28825527191
R2: -0.16787263529144325
Fold:  5
MAPE: 0.22431295869805995
RMSE: 18677.90670937059
R2: -4.405737810807202
Fold:  6
MAPE: 0.2626545602454301
RMSE: 19862.757357528317
R2: -6.185982988155362
Fold:  7
MAPE: 0.2380247147325708
RMSE: 18265.077491909095
R2: -4.874061119167499
Fold:  8
MAPE: 0.22032981880946667
RMSE: 17076.395642810894
R2: -4.091076194360116
Mean MAPE across folds for current weights: 0.17295455687757394
Current Weights: [0.39130435 0.26086957 0.34782609]

Fold:  1
MAPE: 0.10431059014962056
RMSE: 7775.742810929304
R2: 0.5496565405665983
Fold:  2
MAPE: 0.06366994683555893
RMSE: 4002.277173099397
R2: 0.8364835433412284
Fold:  3
MAPE: 0.1040770320461839
RMSE: 10090.69399597037
R2: 0.5426145008418795
Fold:  4
MAPE: 0.17003373988558054
RMSE: 15073.703970252716
R2: -0.18011901792372398
Fold:  5
MAPE: 0.2277202269020236
RMSE: 18

### Test set

In [30]:

# Initialize lists to store MAPE values for each fold
mape_test_folds = []
rmse_test_folds = []
r2_test_folds = []
best_combined_test_forecasts = []
actuals_test = []

# Loop over test splits
for idx, (train_data, test_data) in enumerate(zip(splits_test['train'], splits_test['test'])):
    print(f"Test Split {idx+1}:")

    history = [x for x in train_data]
    pred_1 = []
    pred_2 = []
    pred_3 = []
    actual = []

    for i in range(len(test_data)):
        obs = test_data.values[i]
        actual.append(obs)

    for i in range(-4, 0, 1):
        p_1 = history[i]
        p_2 = history[i - 4]
        p_3 = history[i - 8]
        pred_1.append(p_1)
        pred_2.append(p_2)
        pred_3.append(p_3)

    forecast_lists = np.vstack((pred_1, pred_2, pred_3))

    # Apply the best weights obtained from validation splits
    combined_forecast_test = np.dot(best_weights_overall, forecast_lists)

    best_combined_test_forecasts.append(combined_forecast_test)
    actuals_test.append(test_data.values)


    # Calculate metrics for the test set
    best_rmse_test = sqrt(mean_squared_error(actual, combined_forecast_test))
    best_mape_test = mean_absolute_percentage_error(actual, combined_forecast_test)
    best_r2_test = r2_score(actual, combined_forecast_test)

    print("MAPE of test set:", round(best_mape_test, 4))
    print("RMSE of test set:", round(best_rmse_test, 4))
    print("R2 of test set:", round(best_r2_test, 4))
    print("Best Weights of test set:", best_weights_overall)
    print("")

    # Append MAPE for this fold to the list
    mape_test_folds.append(best_mape_test)
    rmse_test_folds.append(best_rmse_test)
    r2_test_folds.append(best_r2_test)



# Calculate the mean MAPE over all folds
mean_mape_test_overall = np.mean(mape_test_folds)
mean_std_test_overall = np.std(mape_test_folds)
mean_rmse_test_overall = np.mean(rmse_test_folds)
mean_r2_test_overall = np.mean(r2_test_folds)

print("Overall Mean MAPE across all test splits:", mean_mape_test_overall)
print("Overall MAPE std across all test splits:", mean_std_test_overall)
print("Overall Mean RMSE across all test splits:", mean_rmse_test_overall)
print("Overall Mean R2 across all test splits:", mean_r2_test_overall)

Test Split 1:
MAPE of test set: 0.2169
RMSE of test set: 17159.0901
R2 of test set: -3.5623
Best Weights of test set: [0.83333333 0.08333333 0.08333333]

Test Split 2:
MAPE of test set: 0.241
RMSE of test set: 18309.4633
R2 of test set: -5.106
Best Weights of test set: [0.83333333 0.08333333 0.08333333]

Test Split 3:
MAPE of test set: 0.2044
RMSE of test set: 16659.6125
R2 of test set: -3.8868
Best Weights of test set: [0.83333333 0.08333333 0.08333333]

Test Split 4:
MAPE of test set: 0.1532
RMSE of test set: 12630.405
R2 of test set: -1.7852
Best Weights of test set: [0.83333333 0.08333333 0.08333333]

Test Split 5:
MAPE of test set: 0.1606
RMSE of test set: 11213.9892
R2 of test set: 0.4569
Best Weights of test set: [0.83333333 0.08333333 0.08333333]

Test Split 6:
MAPE of test set: 0.1244
RMSE of test set: 8934.9514
R2 of test set: 0.6386
Best Weights of test set: [0.83333333 0.08333333 0.08333333]

Test Split 7:
MAPE of test set: 0.1431
RMSE of test set: 9751.6711
R2 of test set:

In [31]:
# Calculate absolute percentage errors
absolute_percentage_errors_wn = []

for preds, actual in zip(best_combined_test_forecasts, actuals_test):
    abs_percent_errors = [abs((pred - true) / true) for pred, true in zip(preds, actual)]
    absolute_percentage_errors_wn.append(abs_percent_errors)

In [32]:
mean_values_wn = [np.mean(member) for member in zip(*absolute_percentage_errors_wn)]
mean_values_wn

[0.18873723128298953,
 0.18269765563726237,
 0.17376770384391155,
 0.13919775434044432]

### Results

In [33]:
wnaive_values = {
    'wNaive valid': [round(best_mape_overall, 4), round(best_std_overall, 4), round(best_rmse_overall, 4), round(best_r2_overall, 4), '***'],
    'wNaive test': [round(mean_mape_test_overall, 4), round(mean_std_test_overall, 4), round(mean_rmse_test_overall, 4), round(mean_r2_test_overall, 4), mean_values_wn]
}

# Update the existing DataFrame with the new column
df_results['wNaive valid'] = wnaive_values['wNaive valid']
df_results['wNaive test'] = wnaive_values['wNaive test']

# Display the updated DataFrame
df_results

,sNaive valid,sNaive test,wNaive valid,wNaive test
Mean MAPE,0.1434,0.1721,0.148,0.1711
MAPE std,0.0633,0.0372,0.0653,0.0412
Mean RMSE,12002.2089,13204.6307,12630.405,12937.8088
Mean R2,-1.4037,-1.4762,-1.7852,-1.5385
FH MAPEs,***,"[0.1878773278197889, 0.18045408841835758, 0.17...",***,"[0.18873723128298953, 0.18269765563726237, 0.1..."


In [34]:
df_results.to_csv('baseline_results_v2.csv')